In [1]:
import csv
import numpy as np
import matplotlib.pyplot as plt

In [2]:
cost_model_file = '../one-tm-seed-1-pods-4-data/costs_fat_tree_4_pods_1.csv'
opt_value_file = '../values_target.csv'
learned_value_file = '../values_model_estimated.csv'
max_num_steps = 4
switch_to_update = set([0, 1, 2, 3, 4, 5, 8, 9, 12, 13, 16, 17])

In [23]:
def read_values(file_path):
    values = [[] for _ in range(max_num_steps)]
    switch_idx = [[] for _ in range(max_num_steps)]
    switch_idx_map = [{} for _ in range(max_num_steps)]

    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            val = float(row[0])
            num_steps_left = int(row[1]) - 1  # for idx
            
            try:
                switch = [int(i) for i in row[2:]]
            except ValueError:
                switch = []
            
            values[num_steps_left].append(val)
            switch_idx[num_steps_left].append(switch)
            switch_idx_map[num_steps_left][tuple(sorted(switch))] = val

    return values, switch_idx, switch_idx_map

In [24]:
def read_cost_model(file_path):
    cost_model = {}  # switch_down (sorted) -> cost
    
    lc = 0
    with open(file_path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        for row in csv_reader:
            if lc == 0:
                lc += 1
                continue
                
            cost = float(row[0])
            switch_down = [int(i) for i in row[1:]]
            
            switch_key = tuple(sorted(switch_down))
            cost_model[switch_key] = cost

    return cost_model

In [26]:
cost_model = read_cost_model(cost_model_file)
opt_values, opt_switch_idx, opt_switch_map = read_values(opt_value_file)
learned_values, learned_switch_idx, learned_switch_map = read_values(learned_value_file)

In [30]:
if () not in cost_model:
    cost_model[()] = 0

In [36]:
switches_left = set(switch_to_update)
cost_so_far = 0

for i in range(max_num_steps, 0, -1):
    print('Updating switches {} in {} steps'.format(
      switches_left, i))

    step_idx = i - 1
    
    opt_val = opt_switch_map[step_idx][tuple(sorted(switches_left))]
    learned_val = learned_switch_map[step_idx][tuple(sorted(switches_left))]
    
    print(('Step {} left, optimal total value: {}, ' + 
               'learned predicted total value: {}').format(
              i, opt_val, learned_val))
    
    # plan ahead one step
    if i > 1:
        cost = np.inf
        min_next_val = np.inf
        switches_down = set()
        for (nv, sl) in zip(learned_values[step_idx - 1], 
                            learned_switch_idx[step_idx - 1]):

            # current switch left should contain next one
            if len(set(sl) - switches_left) == 0:
                sd = switches_left - set(sl)
                c = cost_model[tuple(sorted(sd))]
                if nv + c < min_next_val:
                    cost = c
                    min_next_val = nv + c
                    switches_down = sd
        
        cost_so_far += cost
        
        print('Switches to take down: {}, cost + future value: {} + {}, cost so far {}'.format(
              switches_down, cost, min_next_val - cost, cost_so_far))
        
        # take down these switches
        switches_left -= switches_down
    
    else:
        
        cost = cost_model[tuple(sorted(switches_left))]
        cost_so_far += cost
        print('Switches to take down: {}, cost {}, cost so far {}'.format(
            switches_left, cost, cost_so_far))
        
    
    
#     opt_i = np.argmin(opt_values[step_idx])
#     learned_i = np.argmin(learned_values[step_idx])
    
#     print(('Step {} left, optimal value: {}, ' + 
#                'optimal switch down: {}').format(
#               i, opt_values[step_idx][opt_i],
#               opt_switch_idx[step_idx][opt_i]))

#     if i == 1:
#         # get cost
#         if len(switches_left) == 0:
#             cost = 0
#         else:
#             cost = cost_model[tuple(sorted(switches_left))]
            
#         cost_so_far += cost

#         print(('Step {} left, learned value: {}, ' +
#                'switches to update: {}, cost: {}, ' + 
#                'cost so far: {}').format(
#               i, learned_values[step_idx][learned_i],
#               learned_switch_idx[step_idx][learned_i],
#               cost, cost_so_far))

#     else:
#         # plan for the next step
#         planned_value = []
#         for (nv, sl) in zip(learned_values[step_idx - 1], 
#                             learned_switch_idx[step_idx - 1]):
#             if len(set(sl) - switches_left) > 0:
#                 planned_value.append(np.inf)  # switch to update again, no
#             else:
#                 sd = switches_left - set(sl)
#                 c = cost_model[tuple(sorted(sd))]
#                 planned_value.append(c + nv)
        
        
#         l_next_i = np.argmin(planned_value[step_idx - 1])
#         switches_down = switches_left - set(
#             learned_switch_idx[step_idx - 1][l_next_i])
        
#         # get cost 
#         cost = cost_model[tuple(sorted(switches_down))]
#         cost_so_far += cost
#         switches_left -= switches_down
        
#         # report
#         print(('Step {} left, learned value: {}, ' + 
#                'switches to update {}, ' +
#                'cost + next value: {} + {}, ' + 
#                'cost so far: {}').format(
#               i, learned_values[step_idx][learned_i],
#               learned_switch_idx[step_idx][learned_i],
#               cost, learned_values[step_idx - 1][l_next_i],
#               cost_so_far))
        

Updating switches {0, 1, 2, 3, 4, 5, 8, 9, 12, 13, 16, 17} in 4 steps
Step 4 left, optimal total value: 0.3, learned predicted total value: 0.2966504
Switches to take down: {0}, cost + future value: 0.0 + 0.288481, cost so far 0.0
Updating switches {1, 2, 3, 4, 5, 8, 9, 12, 13, 16, 17} in 3 steps
Step 3 left, optimal total value: 0.3, learned predicted total value: 0.288481
Switches to take down: {3}, cost + future value: 0.0 + 0.2883607, cost so far 0.0
Updating switches {1, 2, 4, 5, 8, 9, 12, 13, 16, 17} in 2 steps
Step 2 left, optimal total value: 0.3, learned predicted total value: 0.2883607
Switches to take down: {1, 4, 8, 12, 16}, cost + future value: 0.15 + 0.36140227, cost so far 0.15
Updating switches {2, 5, 9, 13, 17} in 1 steps
Step 1 left, optimal total value: 0.15, learned predicted total value: 0.36140227
Switches to take down: {2, 5, 9, 13, 17}, cost 0.15, cost so far 0.3
